In [1]:
from chain_builder import get_sentence
import cPickle as pickle
import tweepy
from datetime import date, timedelta
from keys import keys

In [2]:
with open('Pickles/forward_dict.pkl') as f:
    f_dict = pickle.load(f)

with open('Pickles/reversed_dict.pkl') as f:
    r_dict = pickle.load(f)

In [15]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweet_replies = []

today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)

# ******************************************************
# Queries Popular Tweets directed at Candidates or using
# well-known Hashtags and responds to all queried tweets.
# ******************************************************
hashtags = ["#FeelTheBern OR #ImWithHer OR #MakeAmericaGreatAgain OR #Trump2016 \
OR @BernieSanders OR @HillaryClinton OR @realDonaldTrump"]

top_tweets = tweepy.Cursor(api.search, q=hashtags, count=100, result_type='popular', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in top_tweets:
    if status.favorite_count > 1000:
        tweet_replies.append(status)

# ***************************************************
# Queries Tweets directed at or in reply to
# FeelTheBern_Bot and responds to all queried tweets.
# ***************************************************
bernbot_query = "to:FeelTheBern_Bot OR @FeelTheBern_Bot"

bernbot_tweets = tweepy.Cursor(api.search, q=bernbot_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in bernbot_tweets:
    tweet_replies.append(status)

# ************************************************
# Queries Tweets from Presidential Candidates and
# responds to those with greater than 500 likes.
# ************************************************
candidate_query = "from:BernieSanders OR from:HillaryClinton OR from:realDonaldTrump"

candidate_tweets = tweepy.Cursor(api.search, q=candidate_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in candidate_tweets:
    if status.favorite_count > 500:
        tweet_replies.append(status)


# DROPS ANY DUPLICATE TWEETS SO IT DOES NOT RESPOND TWICE
tweet_replies = set(tweet_replies)

In [16]:
best_tweets = []
for i, tweet in enumerate(tweet_replies):
    resp_length = 141
    text = tweet.text
    sn = tweet.user.screen_name
    response = ':'

    while resp_length > 140 and ':' in response:
        response = get_sentence(text, f_dict, r_dict, randomness = 1) 
        response = "@{0} {1}".format(sn, response)
        resp_length = len(response)
    
    print i, '----', tweet.id
    print '@', sn, text
    print response
    print '*************'
    best_tweets.append([response, tweet.id])

0 ---- 727241630097383424
@ BernieSanders Sick and tired of large corporations trying to cut social programs while they're shipping millions of jobs abroad.
https://t.co/jTiGupcrwz
@BernieSanders You said, quote: which personally I think would have been a majority of women on social Security.
*************
1 ---- 727120229591961600
@ BernieSanders I want Wall Street to help kids in this country attend public colleges and universities tuition-free with a Wall Street speculation tax.
@BernieSanders When we begin to emphasize that, I think, the idea of building a wall along the southern border.
*************
2 ---- 726935349973385216
@ HillaryClinton We should be making it easier to vote, not harder. The attacks on voting rights have to end.
@HillaryClinton RUBIO: We're going to end sanctuary cities.
*************
3 ---- 726939248922865664
@ HillaryClinton "Let’s keep going together.
Let’s organize and mobilize.
Let’s make sure that love trumps hate." —Hillary
@HillaryClinton What I said,

In [17]:
best_list = [0,1,3,5,6,11,30,26,39,41]
for i, tweet in enumerate(best_tweets):
    if i in best_list:
        print i 
        print tweet[0], tweet[1]
        s = api.update_status(tweet[0], tweet[1])

0
@BernieSanders You said, quote: which personally I think would have been a majority of women on social Security. 727241630097383424
1
@BernieSanders When we begin to emphasize that, I think, the idea of building a wall along the southern border. 727120229591961600
3
@HillaryClinton What I said, bomb the oil and keep the oil. 726939248922865664
5
@MichaelAdams317 It's obviously very popular in this crowd, and it's also possible to improve how we police our police. 727277123040370688
6
@HillaryClinton And my plan goes so much further to try to prevent what they saw as a mass genocide, in their words. 727226308330029061
11
@realDonaldTrump Here's the thing - it was sort of funny - 65 percent of Republicans think Donald is right. 727225626575421440
26
@DanScavino Need a president who, in the late 90s and in the air both in Syria and Iraq to fight ISIS. 727114876485693440
30
@BernieSanders And I'd pay for that - I'd pay for that, quote, effing wall. 726949851355734016
39
@HillaryClinton A